In [ ]:
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150
import matplotlib.pyplot as plt
import hdbscan
import numpy as np
import scarlet.display

filename = 'frt00003e12_07_if166j_ter3'

hdr_file = filename + '.hdr'
lbl_file = filename + '.lbl'
img_file = filename + '.img'

# load and reshape binary data
data = np.fromfile(img_file, np.float32)   # data format: float 32 bit, from blb file
C, Ny, Nx = 545, 480, 640                  # cube dimensions, from lbl file
cube = data.reshape(C, Ny, Nx)[:,:,::-1]   # flip horizontal

# channel wavelengths, from hdr file
lambdas = np.array([
   364.620000,   371.120000,   377.620000,   384.120000,   390.620000,
   397.120000,   403.620000,   410.120000,   416.620000,   423.120000,
   429.630000,   436.130000,   442.630000,   449.140000,   455.640000,
   462.150000,   468.650000,   475.160000,   481.670000,   488.170000,
   494.680000,   501.190000,   507.700000,   514.210000,   520.720000,
   527.230000,   533.740000,   540.250000,   546.760000,   553.270000,
   559.780000,   566.290000,   572.810000,   579.320000,   585.830000,
   592.350000,   598.860000,   605.380000,   611.890000,   618.410000,
   624.920000,   631.440000,   637.960000,   644.480000,   650.990000,
   657.510000,   664.030000,   670.550000,   677.070000,   683.590000,
   690.110000,   696.630000,   703.160000,   709.680000,   716.200000,
   722.720000,   729.250000,   735.770000,   742.300000,   748.820000,
   755.350000,   761.870000,   768.400000,   774.920000,   781.450000,
   787.980000,   794.510000,   801.040000,   807.560000,   814.090000,
   820.620000,   827.150000,   833.680000,   840.220000,   846.750000,
   853.280000,   859.810000,   866.340000,   872.880000,   879.410000,
   885.950000,   892.480000,   899.020000,   905.550000,   912.090000,
   918.620000,   925.160000,   931.700000,   938.240000,   944.770000,
   951.310000,   957.850000,   964.390000,   970.930000,   977.470000,
   984.010000,   990.550000,   997.100000,  1003.640000,  1010.180000,
  1016.720000,  1023.270000,  1029.810000,  1036.360000,  1042.900000,
  1049.450000,  1055.990000,  1001.350000,  1007.900000,  1014.450000,
  1021.000000,  1027.550000,  1034.100000,  1040.650000,  1047.200000,
  1053.750000,  1060.300000,  1066.850000,  1073.410000,  1079.960000,
  1086.510000,  1093.070000,  1099.620000,  1106.170000,  1112.730000,
  1119.280000,  1125.840000,  1132.390000,  1138.950000,  1145.510000,
  1152.060000,  1158.620000,  1165.180000,  1171.730000,  1178.290000,
  1184.850000,  1191.410000,  1197.970000,  1204.530000,  1211.090000,
  1217.650000,  1224.210000,  1230.770000,  1237.330000,  1243.890000,
  1250.450000,  1257.010000,  1263.570000,  1270.140000,  1276.700000,
  1283.260000,  1289.830000,  1296.390000,  1302.950000,  1309.520000,
  1316.080000,  1322.650000,  1329.210000,  1335.780000,  1342.340000,
  1348.910000,  1355.480000,  1362.050000,  1368.610000,  1375.180000,
  1381.750000,  1388.320000,  1394.890000,  1401.450000,  1408.020000,
  1414.590000,  1421.160000,  1427.730000,  1434.310000,  1440.880000,
  1447.450000,  1454.020000,  1460.590000,  1467.160000,  1473.740000,
  1480.310000,  1486.880000,  1493.460000,  1500.030000,  1506.610000,
  1513.180000,  1519.760000,  1526.330000,  1532.910000,  1539.480000,
  1546.060000,  1552.640000,  1559.210000,  1565.790000,  1572.370000,
  1578.950000,  1585.520000,  1592.100000,  1598.680000,  1605.260000,
  1611.840000,  1618.420000,  1625.000000,  1631.580000,  1638.160000,
  1644.740000,  1651.330000,  1657.910000,  1664.490000,  1671.070000,
  1677.660000,  1684.240000,  1690.820000,  1697.410000,  1703.990000,
  1710.580000,  1717.160000,  1723.750000,  1730.330000,  1736.920000,
  1743.510000,  1750.090000,  1756.680000,  1763.270000,  1769.850000,
  1776.440000,  1783.030000,  1789.620000,  1796.210000,  1802.800000,
  1809.390000,  1815.980000,  1822.570000,  1829.160000,  1835.750000,
  1842.340000,  1848.930000,  1855.520000,  1862.120000,  1868.710000,
  1875.300000,  1881.900000,  1888.490000,  1895.080000,  1901.680000,
  1908.270000,  1914.870000,  1921.460000,  1928.060000,  1934.650000,
  1941.250000,  1947.850000,  1954.440000,  1961.040000,  1967.640000,
  1974.240000,  1980.840000,  1987.430000,  1994.030000,  2000.630000,
  2007.230000,  2013.830000,  2020.430000,  2027.030000,  2033.630000,
  2040.240000,  2046.840000,  2053.440000,  2060.040000,  2066.640000,
  2073.250000,  2079.850000,  2086.450000,  2093.060000,  2099.660000,
  2106.270000,  2112.870000,  2119.480000,  2126.080000,  2132.690000,
  2139.300000,  2145.900000,  2152.510000,  2159.120000,  2165.720000,
  2172.330000,  2178.940000,  2185.550000,  2192.160000,  2198.770000,
  2205.380000,  2211.990000,  2218.600000,  2225.210000,  2231.820000,
  2238.430000,  2245.040000,  2251.650000,  2258.270000,  2264.880000,
  2271.490000,  2278.100000,  2284.720000,  2291.330000,  2297.950000,
  2304.560000,  2311.180000,  2317.790000,  2324.410000,  2331.020000,
  2337.640000,  2344.260000,  2350.870000,  2357.490000,  2364.110000,
  2370.720000,  2377.340000,  2383.960000,  2390.580000,  2397.200000,
  2403.820000,  2410.440000,  2417.060000,  2423.680000,  2430.300000,
  2436.920000,  2443.540000,  2450.170000,  2456.790000,  2463.410000,
  2470.030000,  2476.660000,  2483.280000,  2489.900000,  2496.530000,
  2503.120000,  2509.720000,  2516.320000,  2522.920000,  2529.510000,
  2536.110000,  2542.710000,  2549.310000,  2555.910000,  2562.510000,
  2569.110000,  2575.710000,  2582.310000,  2588.910000,  2595.510000,
  2602.120000,  2608.720000,  2615.320000,  2621.920000,  2628.530000,
  2635.130000,  2641.740000,  2648.340000,  2654.950000,  2661.550000,
  2668.160000,  2674.760000,  2681.370000,  2687.980000,  2694.580000,
  2701.190000,  2760.680000,  2767.290000,  2773.900000,  2780.520000,
  2787.130000,  2793.740000,  2800.350000,  2806.970000,  2813.580000,
  2820.200000,  2826.810000,  2833.430000,  2840.040000,  2846.660000,
  2853.280000,  2859.890000,  2866.510000,  2873.130000,  2879.750000,    
  2886.360000,  2892.980000,  2899.600000,  2906.220000,  2912.840000,
  2919.460000,  2926.080000,  2932.700000,  2939.320000,  2945.950000,
  2952.570000,  2959.190000,  2965.810000,  2972.440000,  2979.060000,
  2985.680000,  2992.310000,  2998.930000,  3005.560000,  3012.180000,
  3018.810000,  3025.440000,  3032.060000,  3038.690000,  3045.320000,
  3051.950000,  3058.570000,  3065.200000,  3071.830000,  3078.460000,
  3085.090000,  3091.720000,  3098.350000,  3104.980000,  3111.610000,
  3118.250000,  3124.880000,  3131.510000,  3138.140000,  3144.780000,
  3151.410000,  3158.040000,  3164.680000,  3171.310000,  3177.950000,
  3184.580000,  3191.220000,  3197.850000,  3204.490000,  3211.130000,
  3217.760000,  3224.400000,  3231.040000,  3237.680000,  3244.320000,
  3250.960000,  3257.600000,  3264.240000,  3270.880000,  3277.520000,
  3284.160000,  3290.800000,  3297.440000,  3304.080000,  3310.730000,
  3317.370000,  3324.010000,  3330.660000,  3337.300000,  3343.950000,
  3350.590000,  3357.240000,  3363.880000,  3370.530000,  3377.170000,
  3383.820000,  3390.470000,  3397.120000,  3403.760000,  3410.410000,
  3417.060000,  3423.710000,  3430.360000,  3437.010000,  3443.660000,
  3450.310000,  3456.960000,  3463.610000,  3470.260000,  3476.920000,
  3483.570000,  3490.220000,  3496.870000,  3503.530000,  3510.180000,
  3516.840000,  3523.490000,  3530.150000,  3536.800000,  3543.460000,
  3550.110000,  3556.770000,  3563.430000,  3570.080000,  3576.740000,
  3583.400000,  3590.060000,  3596.720000,  3603.380000,  3610.040000,
  3616.700000,  3623.360000,  3630.020000,  3636.680000,  3643.340000,
  3650.000000,  3656.670000,  3663.330000,  3669.990000,  3676.650000,
  3683.320000,  3689.980000,  3696.650000,  3703.310000,  3709.980000,
  3716.640000,  3723.310000,  3729.980000,  3736.640000,  3743.310000,
  3749.980000,  3756.650000,  3763.310000,  3769.980000,  3776.650000,
  3783.320000,  3789.990000,  3796.660000,  3803.330000,  3810.000000,
  3816.670000,  3823.350000,  3830.020000,  3836.690000,  3843.360000,
  3850.040000,  3856.710000,  3863.390000,  3870.060000,  3876.730000,
  3883.410000,  3890.080000,  3896.760000,  3903.440000,  3910.110000,
  3916.790000,  3923.470000,  3930.150000,  3936.820000, 65535.000000    
])

# band channels, from hdr file
bll = np.array([
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0    
], dtype='bool')

# remove junk: noisy/corrupt frequencies and pixels
# min_c, max_c = 0, 545 # 5, 360 # 540
min_y, max_y = 1, 479
min_x, max_x = 9, 608
cube = cube[bll, min_y:max_y, min_x:max_x]
cube[cube == 65535.] = 0
lambdas = lambdas[bll]

In [ ]:
plt.plot(lambdas, cube.mean(axis=(1,2)))
plt.xlabel(r'$\lambda$ [nm]');

In [ ]:
# for visualization only: split into a few channels and make rgb image
def crism_to_rgb(cube, channels=3, standardize=True, norm=None):
    channels = np.stack([ c.sum(axis=0) for c in np.array_split(cube, channels) ], axis=0)
    # make uniform across channels
    if standardize:
        channels -= channels.mean(axis=(1,2))[:,None,None]
        channels /= channels.std(axis=(1,2))[:,None,None]
    if norm is None:
        norm = scarlet.display.AsinhPercentileNorm(channels)
    return scarlet.display.img_to_rgb(channels, norm=norm)

plt.imshow(crism_to_rgb(cube))
plt.gca().axis('off');

In [ ]:
# intensity normalization
cube_normed = cube / cube.sum(axis=0)[None, :, :]

plt.imshow(crism_to_rgb(cube_normed, channels=6))
plt.gca().axis('off');